# Times obtained during analysis test (3 Orgs - 1 Peer Node per Org)

### Imports

In [1]:
import sys
sys.path.insert(1, '..\code')
from analyst import *
import threading
import requests
import json
import random
import plotly
import plotly.graph_objs as go

### Global variables

In [2]:
API_ENDPOINT = "http://34.76.123.255:3000/api/"
NS = "ertis.uma.nuclear"

### Function

In [ ]:
def addAnalysisTest(num_analysts, num_acqs):
    analyst_threads = []
    analysis_per_worker = int(num_acqs/num_analysts)
    
    # Check for next analysis id
    resource_url = f"{API_ENDPOINT}{NS}.Analysis"
    r = requests.get(resource_url)
    next_id = len(r.json()) + 1
    
    # Create threads
    for i in range(num_analysts):
        thread = Analyst(i, f"Analyst-{i}", analysis_per_worker, next_id+analysis_per_worker*i, API_ENDPOINT, NS)
        analyst_threads.append(thread)
        
    # Start threads
    for i in range(num_analysts):
        analyst_threads[i].start()
        
    # Join threads
    for i in range(num_analysts):
        analyst_threads[i].join()
        
    # Print results
    min_get = analyst_threads[0].min_get
    min5avg_get = analyst_threads[0].min5avg_get
    avg_get = analyst_threads[0].avg_get
    max_get = analyst_threads[0].max_get
    max5avg_get = analyst_threads[0].max5avg_get
    
    min_add = analyst_threads[0].min_add
    min5avg_add = analyst_threads[0].min5avg_add
    avg_add = analyst_threads[0].avg_add
    max_add = analyst_threads[0].max_add
    max5avg_add = analyst_threads[0].max5avg_add
    
    analyst_threads[0].printResults()
    for i in range(1, num_analysts):
        if analyst_threads[i].min_get < min_get:
            min_get = analyst_threads[i].min_get
        
        min5avg_get = min5avg_get + acquisitor_threads[i].min5avg_get

        avg_get = avg_get + analyst_threads[i].avg_get

        if analyst_threads[i].max_get > max_get:
            max_get = analyst_threads[i].max_get
            
        max5avg_get = max5avg_get + acquisitor_threads[i].max5avg_get
            
        if analyst_threads[i].min_add < min_add:
            min_add = analyst_threads[i].min_add
            
        min5avg_add = min5avg_add + acquisitor_threads[i].min5avg_add
            
        avg_add = avg_add + analyst_threads[i].avg_add
        
        if analyst_threads[i].max_get > max_get:
            max_add = analyst_threads[i].max_add
        
        max5avg_add = max5avg_add + acquisitor_threads[i].max5avg_add

        analyst_threads[i].printResults()

    print(f"MIN GET ACQ --> {min_get}")
    print(f"MIN 5 GET ACQ --> {min5avg_get/num_analysts}")
    print(f"AVG GET ACQ --> {avg_get/num_analysts}")
    print(f"MAX GET ACQ --> {max_get}")
    print(f"MAX 5 GET ACQ --> {max5avg_get/num_analysts}")
    
    print(f"MIN ADD ANA --> {min_add}")
    print(f"MIN 5 GET ANA --> {min5avg_add/num_analysts}")
    print(f"AVG ADD ANA --> {avg_add/num_analysts}")
    print(f"MAX ADD ANA --> {max_add}")
    print(f"MAX 5 GET ANA --> {max5avg_add/num_analysts}")

    print("Add Analysis Test Finalized")